<h1><center>Laboratorio 6: Optimización de modelos 🧪</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2025</strong></center>

### Cuerpo Docente:

- Profesores: Diego Cortez, Gabriel Iturra
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Nicolás Cabello, Cristopher Urbina

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: María Jesús Valenzuela
- Nombre de alumno 2: Matías Zúñiga

Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda **fuertemente** asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/jesuvalenzuela/NanSquad)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.


### Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda fuertemente asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [83]:
# !pip install -qq xgboost optuna
# !pip install optuna-integration[xgboost]

import time
from datetime import datetime

import numpy as np
import pandas as pd
import joblib
import optuna
import optuna.visualization as vis

from sklearn import set_config
set_config(transform_output="pandas")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error

from xgboost import XGBRegressor
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
from optuna.integration import XGBoostPruningCallback
from optuna.importance import get_param_importances

optuna.logging.set_verbosity(optuna.logging.WARNING)


# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [84]:
# Importar datos
df = pd.read_csv('sales.csv', 
                 parse_dates = ['date'],
                 date_format="%d/%m/%y", 
                 index_col='id')

df.head()

,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
id,,,,,,,,,,,
0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [85]:
# Inserte su código acá

# === 1. Separar los datos en conjuntos de train (70%), validation (20%) y test (10%).
X = df.drop(columns=['quantity'])
y = df['quantity']

# Primero separamos conjunto de test
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)

# Luego separamos los datos restantes en entrenamiento y validación
val_size = 0.2 / 0.9        # proporción de datos restantes que representan el 20% del total
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=val_size, random_state=42)


# === 2. Implementar un `FunctionTransformer` para extraer el día, mes y año de la variable `date`.

def extract_date_features(X):
    """
    Extrae día, mes y año de la columna 'date' y los convierte a categorical.
    """
    X_copy = X.copy()
    
    # Asegurar formato datetime
    if not pd.api.types.is_datetime64_any_dtype(X_copy['date']):
        X_copy['date'] = pd.to_datetime(X_copy['date'])
    
    # Extraer características temporales
    X_copy['day'] = X_copy['date'].dt.day.astype('category')
    X_copy['month'] = X_copy['date'].dt.month.astype('category')
    X_copy['year'] = X_copy['date'].dt.year.astype('category')
    
    # Eliminar columna original
    X_copy = X_copy.drop(columns=['date'])
    
    return X_copy


# Crear FunctionTransformer
date_transformer = FunctionTransformer(extract_date_features)


# === 3. Implementar un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos.

class AutoColumnTransformer(BaseEstimator, TransformerMixin):
    """
    ColumnTransformer que identifica automáticamente variables numéricas y categóricas.
    
    Durante fit(), identifica los tipos de columnas y crea internamente un 
    ColumnTransformer apropiado. Esto evita tener que especificar columnas manualmente.
    """
    
    def __init__(self, min_frequency=None):
        """
        Inicializa el transformador.
        """
        self.min_frequency = min_frequency
        self.column_transformer_ = None
    
    def fit(self, X, y=None):
        """
        Identifica columnas numéricas y categóricas, y crea el ColumnTransformer.
        """
        # Identificar tipos de columnas
        numeric_features = X.select_dtypes(include='number').columns.tolist()
        categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
        
        # Crear ColumnTransformer
        self.column_transformer_ = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), numeric_features),
                ('cat', OneHotEncoder(
                    drop='first',
                    sparse_output=False,
                    min_frequency=self.min_frequency        # Para la sección de optimización con optuna
                    ), categorical_features)
                ],
            verbose_feature_names_out=False
            )
        
        # Configurar salida como DataFrame
        self.column_transformer_.set_output(transform='pandas')
        
        # Ajustar el ColumnTransformer
        self.column_transformer_.fit(X, y)
        
        return self
    
    def transform(self, X):
        """
        Aplica la transformación.
        """
        return self.column_transformer_.transform(X)

# Instanciar el transformador    
col_transformer = AutoColumnTransformer()

# === 4. Guardar los pasos anteriores en un `Pipeline`
pipeline = Pipeline(steps=[
    ('date_features', date_transformer),
    ('preprocessor', col_transformer),
    ('regressor', DummyRegressor(strategy='mean'))
])

# === 5. Entrenar el pipeline y reportar la métrica `mean_absolute_error` sobre los datos de validación.

pipeline.fit(X_train, y_train)
y_val_pred = pipeline.predict(X_val)

print(f"MAE validación Dummy = {mean_absolute_error(y_val, y_val_pred):.4f}")

# === 6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. 

pipeline_xgb = Pipeline(steps=[
    ('date_features', date_transformer),
    ('preprocessor', col_transformer),
    ('regressor', XGBRegressor(random_state=42))
])

pipeline_xgb.fit(X_train, y_train)
y_val_pred_xgb = pipeline_xgb.predict(X_val)

print(f"MAE validación XGB = {mean_absolute_error(y_val, y_val_pred_xgb):.4f}")

# === 7. Guardar ambos modelos en un archivo .pkl

# Guardar el pipeline con DummyRegressor
joblib.dump(pipeline, 'pipeline_dummy.pkl')

# Guardar el pipeline con XGBRegressor
joblib.dump(pipeline_xgb, 'pipeline_xgb.pkl')

print("Modelos guardados")

MAE validación Dummy = 13543.9614
MAE validación XGB = 2591.5955
Modelos guardados


5. **¿Cómo se interpreta esta métrica para el contexto del negocio?**

Para el contexto del negocio, el mean absolute error (MAE), indica, la diferencia promedio entre las ventas estimadas y las reales. De manera más literal, el MAE indica la media de los errores absolutos de la estimación. 

6. **¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?**

El MAE disminuye considerablemente con XGBRegressor, en comparación al modelo Dummy. Esto significa una mejora significativa en el desempeño predictivo.


## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [86]:
# Inserte su código acá

# === 1. Pipeline con restricciones monotónicas
pipeline_xgb2 = Pipeline(steps=[
    ('date_features', date_transformer),
    ('preprocessor', col_transformer),
    ('regressor', XGBRegressor(
        monotone_constraints={'price': -1},     # relación monótona negativa con precio
        random_state=42
    ))
])

# Entrenar y predecir
pipeline_xgb2.fit(X_train, y_train)
y_val_pred_xgb2 = pipeline_xgb2.predict(X_val)

# === 2. Reportar MAE
print(f"MAE validación XGB monótono = {mean_absolute_error(y_val, y_val_pred_xgb2):.4f}")

# === 4. Guardar modelo
joblib.dump(pipeline_xgb2, 'pipeline_xgb_monotono.pkl')

print("Modelo guardado")

MAE validación XGB monótono = 2581.0237
Modelo guardado


3. **¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?**

El error disminuye levemente al incluir esta relación, por lo que el amigo tenía razón. Imponer una relación monótona negativa entre price y quantity puede ayudar si esa relación realmente existe en los datos

## 3. Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [87]:
# Inserte su código acá

# === 1. Implementar una función `objective()` que permita minimizar el `MAE` en el conjunto de validación.

# No consideramos 'monotone_constraints', ya que NO es "la mejor configuración obtenida en la sección anterior" 

def objective(trial):
    """
    Función objetivo para optimización con Optuna.
    Minimiza el MAE en el conjunto de validación.
    """

    # Parámetros XBGRegressor
    params_xgb = {
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
            'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'max_leaves': trial.suggest_int('max_leaves', 0, 100),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
            'random_state': 42
        }

    # Parámetros OneHotEncoder
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    # Pipeline
    pipeline = Pipeline(steps=[
        ('date_features', date_transformer),
        ('preprocessor', AutoColumnTransformer(min_frequency=min_frequency)),
        ('regressor', XGBRegressor(**params_xgb))
    ])

    # Entrenar el pipeline
    pipeline.fit(X_train, y_train)
    
    # Predecir en validación
    y_pred_val = pipeline.predict(X_val)
    
    # Calcular MAE
    mae = mean_absolute_error(y_val, y_pred_val)
    
    # Almacenar el mejor pipeline
    trial.set_user_attr('pipeline', pipeline)
    
    return mae


# Configurar el estudio de Optuna
study = optuna.create_study(
    direction='minimize',       # Minimizar MAE
    sampler=TPESampler(seed=42),
    study_name='xgboost_optimization'
)

# === 2. Fijar el tiempo de entrenamiento a 5 minutos (300 segundos)
start_time = time.time()

study.optimize(
    objective,
    timeout=300,
    show_progress_bar=True
)

end_time = time.time()
elapsed_time = end_time - start_time

# Corroborar tiempo
print(f"Tiempo total: {elapsed_time/60:.2f} minutos")

# === 3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados.

# Número de trials
print(f"Número de trials completados: {len(study.trials)}")

# Mejor MAE
print(f"\nMejor MAE en validación: {study.best_value:.4f}")

# Mejores hiperparámetros
print(f"\nMejores hiperparámetros encontrados:")
for param, value in study.best_params.items():
    print(f"  {param:20s}: {value}")

# === 5. Guardar su modelo en un archivo .pkl

# Recuperar el mejor pipeline
best_pipeline = study.best_trial.user_attrs['pipeline']

# Guardar
joblib.dump(best_pipeline, 'pipeline_xgb_optuna.pkl')

print("Modelo guardado")


   0%|          | 00:00/05:00

Tiempo total: 5.04 minutos
Número de trials completados: 123

Mejor MAE en validación: 2079.5949

Mejores hiperparámetros encontrados:
  learning_rate       : 0.07370326670914756
  n_estimators        : 903
  max_depth           : 6
  max_leaves          : 76
  min_child_weight    : 5
  reg_alpha           : 0.23756349049751677
  reg_lambda          : 0.23175623264032388
  min_frequency       : 0.030266493618148038
Modelo guardado


3. **¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?**

Los resultados mejoran respecto a la sección anterior, disminuyendo el MAE en alrededor de 570 unidades respecto al modelo XGB base y 560 respecto al modelo del punto 2. Esto puede explicarse en base a dos factores.

En primer lugar, algunos elementos del modelo óptimo mejoran el proceso de aprendizaje en comparación al modelo original. Por ejemplo, se tiene ``n_estimators = 903`` (vs default 100), donde la mayor cantidad de árboles genera un modelo más estable, ya que se obtienen más "votantes". Luego, ``max_depth = 6`` (vs default 6), genera árboles ligeramente más profundos que logran capturar patrones algo más complejos. Luego, ``reg_lambda = 0.2317`` (vs default 1) disminuye ligeramente el grado de regularización L2, permitiendo coeficientes ligeramente más grandes, lo que reduce la "savidad" del ajuste.

En segundo lugar, el modelo óptimo cuenta con diversos que ayudan a prevenir el *overfitting* y mejorar la generalización del modelo. 
Por ejemplo, ``max_leaves = 76`` (vs default 0) introduce un límite en la cantidad de hojas.
También, ``reg_alpha =  0.2375`` (vs default 0) introduce regularización L1 moderada, lo que permite al modelo, de ser necesario, realizar selección de atributos e ignorar aquellos que no aporten mucho. 
Luego, ``min_child_weight = 5`` (vs default 1), previene que se hagan divisiones que dejen pocas observaciones aisladas, lo que generaría *overfitting*.
Además, ``learning_rate = 0.073`` (vs default 0.3), lo que disminuye la contribución de cada nuevo árbol, evitando que el modelo se "desvíe" demasiado si algún árbol se ajusta a una particularidad de los datos de entrenamiento.
Por último, ``min_frequency = 0.03`` (vs default None) reduce ruido asociado a categorías poco frecuentes/representativas ( <2.70%) al hacer encoding, disminuyendo la dimensionalidad del modelo y mejorando la generalización.

4. **Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?**

- 'learning_rate' (0.001, 0.1): Controla cuánto contribuye cada árbol al resultado final. El rango de optimización es conservador, fomentando un entrenamiento "lento" con más árboles, ayudando a prevenir el sobreajuste.
- 'n_estimators' (50, 1000): Número de árboles. Con más árboles, se obtiene un modelo más robusto y estable, mientras que con menos árboles se obtiene un modelo más sencillo. El límite inferior es razonable en caso de haber patrones sencillos en los datos, aunque el límite superior podría ser bajo para valores bajos de learning_rate, que tienden a necesitar más árboles.
- 'max_depth' (3, 10): Limita la profundidad máxima de cada árbol. Rango razonable, ligeramente por bajo y sobre el valor default. Valores bajo 3 serían árboles demasiado simples para la dimensionalidad del dataset, especialmente usando OneHotEncoder, mientras que sobre 10 podrían sobreajustar.
- 'max_leaves' (0, 100): Número máximo de nodos hoja por árbol. El caso mínimo corresponde al default (no se limitan hojas) lo que es razonable dado que también se restringe 'max_depth'; el máximo es difícil de interpretar, ya que no se sabe de antemano cuantos "escenarios distintos" presentan los datos. 100 podría ser razonable para "tipos" de compras de bebidas.
- 'min_child_weight' (1, 5): De la documentación, es la "Suma mínima del peso de la instancia (hessiano) necesaria en un hijo". En regresión lineal, puede entenderse como la cantidad mínima de observaciones por nodo hijo. 1 puede ser apropiado si se considera la existencia compras anómalas o particulares (por ejemplo, si se realizó una compra grupal para un evento). 5 es más conservador para prevenir *overfitting*, evitando que el modelo aprenda de estas compras "especiales".
- 'reg_alpha' (0, 1): Peso de regularización L1 (Lasso). El rango inferior genera el caso donde no se usa regularizacón L1, lo cual es razonable. Sin embargo, el rango superior es demasiado conservador, limitando la selección de atributos y, posiblemente, evitando una buena generalizacion. Podría ser aceptado dado que también se consideran otros parámetros de regularización (reg_lambda, max_depth, min_child_weight).
- 'reg_lambda' (0, 1): Peso de regularización L2 (Ridge). El rango inferior genera el caso donde no se usa regularizacón L2, lo cual es razonable. El rango superior es demasiado conservador, limitando el control de *overfitting* y, posiblemente, evitando una buena generalizacion. Podría ser aceptado dado que también se consideran otros parámetros de regularización (reg_alpha, max_depth, min_child_weight).
- 'min_frequency' (0.0, 1.0): Especifica la proporción mínima que debe representar una categoría para producir una columna en el encoding. El rango minimo representa el caso donde no se aplica la restricción, lo cual es razonable. Sin embargo, el valor máximo no es razonable pues, ni cualquier valor muy alto, pues se perdería información sobre clases que pueden ser igualmente muy representativas. Sería raro que la optimización elija un valor alto, por lo que al final es una "perdida de eficiencia".


## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

¿Qué es pruning y cómo impacta?

Pruning (poda temprana) es una técnica que corta (“prunea”) los trials que van mal antes de que terminen, usando el desempeño parcial (métricas intermedias).
Algunos de los impactos esperados son menos tiempo por trial, más trials explorados en el mismo presupuesto de tiempo y, a menudo, mejor o igual MAE porque Optuna prueba más combinaciones prometedoras.

In [88]:
# Inserte su código acá

# === Optuna + Pruning ===
optuna.logging.set_verbosity(optuna.logging.WARNING)

# 1) Transformador auxiliar para asegurar tipos adecuados
def cast_object_to_category(X):
    X = X.copy()
    for col in X.select_dtypes(include='object').columns:
        X[col] = X[col].astype('category')
    return X

cast_transformer = FunctionTransformer(cast_object_to_category)

def make_feature_pipeline(min_frequency):
    """Pipeline de features sin modelo, con casteo, date_features y OneHot."""
    return Pipeline(steps=[
        ("cast_obj", cast_transformer),
        ("date_features", date_transformer),
        ("preprocessor", AutoColumnTransformer(min_frequency=min_frequency)),
    ])

def objective_pruned(trial):
    # === Espacio de búsqueda
    params_xgb = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
        "n_estimators":  trial.suggest_int("n_estimators", 50, 1000),
        "max_depth":     trial.suggest_int("max_depth", 3, 10),
        "max_leaves":    trial.suggest_int("max_leaves", 0, 100),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
        "reg_alpha":     trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda":    trial.suggest_float("reg_lambda", 0.0, 1.0),
        "random_state":  42,
        "eval_metric":   "mae",
        "tree_method":   "hist",
        "n_jobs":        -1,
    }

    min_frequency = trial.suggest_float("min_frequency", 0.0, 1.0)

    # === (A) Preprocesamiento de X_train / X_val
    feat_pipe = make_feature_pipeline(min_frequency)
    feat_pipe.fit(X_train, y_train)
    Xtr_t  = feat_pipe.transform(X_train)
    Xval_t = feat_pipe.transform(X_val)

    # === (B) Callback de pruning + modelo
    pruning_cb = XGBoostPruningCallback(trial, "validation_0-mae")
    reg = XGBRegressor(**params_xgb, callbacks=[pruning_cb])

    # Entrenamiento con datos transformados
    reg.fit(
        Xtr_t, y_train,
        eval_set=[(Xval_t, y_val)],
        verbose=False
    )

    # === (C) Evaluación en validación
    y_pred = reg.predict(Xval_t)
    mae = mean_absolute_error(y_val, y_pred)

    # === (D) Empaquetar pipeline final ya entrenado
    final_pipeline = Pipeline(steps=[
        ("cast_obj", cast_transformer),
        ("date_features", date_transformer),
        ("preprocessor", feat_pipe.named_steps["preprocessor"]),
        ("regressor", reg)
    ])
    trial.set_user_attr("pipeline", final_pipeline)
    return mae

# === Estudio Optuna con pruning
study_pruned = optuna.create_study(
    direction="minimize",
    sampler=TPESampler(seed=42),
    pruner=MedianPruner(n_warmup_steps=5),
    study_name="xgb_optuna_pruned"
)

start = time.time()
study_pruned.optimize(objective_pruned, timeout=300, show_progress_bar=True)
elapsed = (time.time() - start) / 60

print(f"\n Tiempo total: {elapsed:.2f} min")
print(f"Trials completados: {len(study_pruned.trials)}")
print(f"Mejor MAE (val): {study_pruned.best_value:.4f}\n")

print("Mejores hiperparámetros (pruned):")
for k, v in study_pruned.best_params.items():
    print(f"  {k:18s}: {v}")

# Guardar mejor modelo
best_pipeline_pruned = study_pruned.best_trial.user_attrs["pipeline"]
joblib.dump(best_pipeline_pruned, "pipeline_xgb_optuna_pruned.pkl")
print("\nModelo guardado")


   0%|          | 00:00/05:00


 Tiempo total: 5.05 min
Trials completados: 203
Mejor MAE (val): 2010.8607

Mejores hiperparámetros (pruned):
  learning_rate     : 0.09342942470032146
  n_estimators      : 629
  max_depth         : 7
  max_leaves        : 73
  min_child_weight  : 4
  reg_alpha         : 0.9609897581020574
  reg_lambda        : 0.34365648777327495
  min_frequency     : 0.05013269215027011

Modelo guardado


**¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?**

En esta sección (con pruning) el mejor MAE de validación bajó de 2079.59 (sección 3) a 2010.86, es decir, una mejora de ~68.7 puntos. Con el mismo presupuesto de ~5 min, se completaron más trials (≈203 vs 123) porque el pruner cortó temprano configuraciones flojas y Optuna pudo explorar más combinaciones prometedoras. El óptimo también cambió, ya que pasó a max_depth=7 (vs 6) con menos árboles (629 vs 903), regularización mucho más fuerte (reg_alpha≈0.96, reg_lambda≈0.34), min_child_weight=4 (vs 5) y min_frequency≈0.05 (vs 0.03). En conjunto, el pruning permitió hallar un punto con árboles algo más profundos pero mejor regularizados y con encoding más agresivo (filtra categorías raras), lo que reduce sobreajuste en one-hot y explica la mejora del MAE.

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [89]:
# Inserte su código acá
# === Visualizaciones con Optuna ===

# 1) Tomar el estudio en memoria (prioriza el podado)
study_to_plot = None
for name in ("study_pruned", "study"):
    if name in globals() and isinstance(globals()[name], optuna.study.Study):
        study_to_plot = globals()[name]
        break
assert study_to_plot is not None, "No encontré un Study en memoria. Ejecuta antes la Sección 3/4."

print(f"Usando estudio: {study_to_plot.study_name}")

# 2) Gráficos (Plotly)
try:
    fig1 = vis.plot_optimization_history(study_to_plot)
    fig1.update_layout(title="Historial de Optimización (MAE)")
    fig1.show()

    fig2 = vis.plot_parallel_coordinate(
        study_to_plot,
        params=[
            "learning_rate","n_estimators","max_depth","max_leaves",
            "min_child_weight","reg_alpha","reg_lambda","min_frequency"
        ]
    )
    fig2.update_layout(title="Coordenadas Paralelas de Hiperparámetros")
    fig2.show()

    fig3 = vis.plot_param_importances(study_to_plot)
    fig3.update_layout(title="Importancia de Hiperparámetros (fANOVA)")
    fig3.show()

except Exception as e:
    raise RuntimeError(
        "No se pudo renderizar con Plotly."
    ) from e


Usando estudio: xgb_optuna_pruned


4. Desde qué trial se empiezan a observar mejoras notables en sus resultados

La mejora notable comienza en el trial numero 1, en donde el MAE cae bruscamente desde ~9500 a ~5700 y desde ahí, continúa disminuyendo hasta estabilizarse cerca de ~2000. Este quiebre temprano y la posterior meseta son coherentes con una búsqueda bayesiana con pruning, que descarta rápidamente configuraciones pobres y concentra el presupuesto en regiones prometedoras del espacio de hiperparámetros.

5. Tendencias observadas en el gráfico de coordenadas paralelas

En el gráfico de coordenadas paralelas, las combinaciones asociadas a menor MAE muestran learning_rate bajo (≈ 0.01–0.08) junto con muchos árboles (≈ 600–900) y profundidad intermedia (5–8), lo que demuestra un aprendizaje más gradual pero estable. También se ve que reg_alpha moderado–alto (≳ 0.2) y min_frequency bajo (≈ 0.02–0.08) tienden a mejorar el desempeño. L1 ayuda a controlar complejidad/selección de atributos y filtrar categorías raras en el one-hot reduce ruido y dimensionalidad efectiva.

6. Hiperparámetros con mayor importancia para la optimización

Según el gráfico de importancia (fANOVA), los hiperparámetros más influyentes son min_frequency (~0.44), reg_alpha (~0.31) y learning_rate (~0.16); en un segundo plano aparecen reg_lambda (~0.03) y max_depth (~0.03), mientras que n_estimators, max_leaves y min_child_weight muestran efectos marginales (≤ 0.01). Esto refuerza que el control de la codificación categórica y la regularización L1, junto con un ritmo de aprendizaje adecuado, dominan la calidad del modelo.

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [90]:
# Inserte su código acá

# Cargamos los modelos guardados previamente
model_paths = {
    "Baseline (Dummy)": "pipeline_dummy.pkl",
    "XGB base": "pipeline_xgb.pkl",
    "XGB monotone": "pipeline_xgb_monotono.pkl",
    "XGB Optuna": "pipeline_xgb_optuna.pkl",
    "XGB Optuna+Pruning": "pipeline_xgb_optuna_pruned.pkl"
}

mae_val = {}
for name, path in model_paths.items():
    model = joblib.load(path)
    y_pred_val = model.predict(X_val)
    mae_val[name] = mean_absolute_error(y_val, y_pred_val)

# --- 1-2. Convertimos a DataFrame y creamos tabla---
df_mae_val = pd.DataFrame.from_dict(mae_val, orient='index', columns=['MAE_validación']).sort_values(by='MAE_validación')
print("\nTabla resumen de MAE en validación")
print(df_mae_val)

# --- 3. Evaluación en conjunto de test ---
best_model = joblib.load(model_paths[best_model_name])
y_pred_test = best_model.predict(X_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
print(f"\n MAE en conjunto de test para {best_model_name}: {mae_test:.4f}")

# --- 4. Diferencia entre metricas validación y test ---
diff = mae_test - df_mae_val.loc[best_model_name, 'MAE_validación']
print(f"Diferencia test - validación: {diff:.4f}")



Tabla resumen de MAE en validación
                    MAE_validación
XGB Optuna+Pruning     2010.860687
XGB Optuna             2079.594853
XGB monotone           2581.023699
XGB base               2591.595532
Baseline (Dummy)      13543.961388

 MAE en conjunto de test para XGB Optuna+Pruning: 1988.1180
Diferencia test - validación: -22.7427


2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento?

La tabla muestra claramente la mejora progresiva desde el modelo Baseline (MAE ≈ 13543) hasta el XGB Optuna+Pruning (MAE ≈ 2010), con una diferencia considerable respecto a los modelos sin optimización. El modelo con mejor rendimiento en validación es XGB Optuna+Pruning

4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

Existen diferencias leves. El MAE en validación fue de 2010 y en test de 1988. Esto puede ocurrir por variaciones en la distribución de los datos entre validación y test, o porque el modelo generaliza bien. La diferencia es pequeña, lo que indica buena generalización y ausencia de sobreajuste.

# Conclusión

El laboratorio permitió aplicar de manera progresiva distintas estrategias para optimizar modelos predictivos, partiendo desde un baseline simple hasta llegar a un modelo ajustado con Optuna y pruning. Se pudo observar cómo el uso de XGBoost mejoró significativamente el MAE respecto al DummyRegressor y cómo la introducción de restricciones monotónicas mejoró el rendimiento levemente al imponer relaciones estructurales entre variables. Posteriormente, la optimización bayesiana mediante Optuna permitió explorar de manera eficiente el espacio de hiperparámetros, reduciendo aún más el error de validación y destacando la importancia de variables como min_frequency, reg_alpha y learning_rate. Las visualizaciones ofrecieron una comprensión clara del progreso de la optimización, las interacciones entre hiperparámetros y su relevancia relativa. Finalmente, la evaluación sobre el conjunto de test confirmó una buena capacidad de generalización, consolidando la efectividad de las técnicas implementadas.


Exito!
<p align="center">
  <img src="https://i.pinimg.com/originals/55/3d/42/553d42bea9b10e0662a05aa8726fc7f4.gif">
</p>